## Импорт и установка библиотек

In [1]:
#!pip install dbfread # file reader
#!pip install patool # rar unarchiver

In [2]:
import pandas as pd
import patoolib
import os
import numpy as np
import shutil

from dbfread import DBF

In [3]:
def read_dbf(file):
    return pd.DataFrame(iter(DBF(file, encoding='cp866')))

## Разархивирование файлов

### Чтение файлов

- Файл mmyyyy**_B.DBF**, где mmyyyy – отчетный месяц, отчетный год: Данные формы 0409101 по кредитным организациям, раскрывающим свою информацию в Интернет (в соответствии с Указанием Банка России от 27.03.1998 No192-У утратило силу в 2009 году)
- Файл mmyyyy**B1.DBF**, где mmyyyy – отчетный месяц, отчетный год: Данные формы 0409101 по кредитным организациям, раскрывающим свою информацию в Интернет (в соответствии с письмом Банка России от 21.12.2006 No165-Т – в полном объеме и с оговоркой)
- Файл mmyyyy**_N.DBF**, где mmyyyy – отчетный месяц, отчетный год: Перечень кредитных организаций, раскрывающих данные формы 0409101 в Интернет (в
соответствии с Указанием Банка России от 27.03.1998 No192-У утратило силу в 2009 году)
- Файл mmyyyy**N1.DBF**, где mmyyyy – отчетный месяц, отчетный год: Перечень кредитных организаций, раскрывающих данные формы 0409101 в Интернет (в соответствии с письмом Банка России от 21.12.2006 No165-Т – в полном объеме и с оговоркой)
- Файл mmyyyy**_S.DBF**, где mmyyyy – отчетный месяц, отчетный год: Контрольная сумма для проверки корректности передачи информации в соответствии с Указанием Банка России от 27.03.1998 No192-У (утратило силу в 2009 году) (для проверки корректности передачи файла «mmyyyy_B.DBF»):
- Файл mmyyyy**S1.DBF**, где mmyyyy – отчетный месяц, отчетный год: Контрольные суммы для проверки корректности передачи информации в соответствии с письмом Банка России от 21.12.2006 No165-Т (для проверки корректности передачи файла «mmyyyyB1.DBF»)
- Файл NAMES.DBF: План счетов бухгалтерского учета в кредитных организациях


In [4]:
df = pd.DataFrame()

In [5]:
chosen_columns = [
    'REGN', # рег номер
    'PLAN', # глава плана счетов (активы (А), ДУ (Б), внебаланс (В), срочные (Г), депо (Д))
    'NUM_SC', # номер счета второго порядка по Плану счетов бухгалтерского учета в кредитных организациях
    'A_P', # 1 - актив, 2 - пассив
    'IITG', # исходящие остатки "итого", в рублях; для депо в штуках
    'DT', # отчетная дата
    'PRIZ', # признак категории раскрытия информации: 1 – полный объем; 2 – исходящие остатки «в рублях», «ин. вал., драг. металлы», «итого»
    'NAME' # текстовое название счета в плане счетов
]

In [6]:
for filename in os.listdir('data/rars'):
    if not filename.startswith('.'):
        name = filename.split('.')[0]

        patoolib.extract_archive(f'data/rars/{filename}', outdir=f'data/{name}', verbosity=-1)
        folder_contents = os.listdir(f'data/{name}')
        filename = [fname for fname in folder_contents if fname.endswith('B1.DBF')][0]
        frame = read_dbf(f'data/{name}/{filename}')
        names = read_dbf(f'data/{name}/NAMES.DBF')
        frame = frame.merge(names[['NUM_SC', 'NAME']], on='NUM_SC')[chosen_columns]
        df = pd.concat([df, frame])

        shutil.rmtree(f'data/{name}')

INFO patool: ... creating output directory `data/101-20150501'.
INFO patool: ... creating output directory `data/101-20100901'.
INFO patool: ... creating output directory `data/101-20130801'.
INFO patool: ... creating output directory `data/101-20140101'.
INFO patool: ... creating output directory `data/101-20101001'.
INFO patool: ... creating output directory `data/101-20131101'.
INFO patool: ... creating output directory `data/101-20101201'.
INFO patool: ... creating output directory `data/101-20140301'.
INFO patool: ... creating output directory `data/101-20150701'.
INFO patool: ... creating output directory `data/101-20111201'.
INFO patool: ... creating output directory `data/101-20150301'.
INFO patool: ... creating output directory `data/101-20140701'.
INFO patool: ... creating output directory `data/101-20140501'.
INFO patool: ... creating output directory `data/101-20110901'.
INFO patool: ... creating output directory `data/101-20120801'.
INFO patool: ... creating output directo

In [ ]:
df.to_csv('data/df.csv')